# Solid-Fluid Equilibria

This notebook exemplifies solid-fluid equilibria calculations. There are two implemented functions for this purpose: 

- `sle`: Computes the Solid-Liquid equilibria of a given mixture of global composition (Z) at temperature (T) and pressure (P).

- `slle`: Computes the Solid-Liquid-Liquid equilibria of a given mixture of global composition (Z) at temperature (T) and pressure (P).

**Notes**
1. Both functions use a modified multiphase flash that computes equilibrium and stability simultaneously. However, note that `sle` doesn't check if the liquid phase is miscible.

2. Both functions consider the solid phases to be pure phases.

3. The function requires you to set up the components' fusion enthalpy and temperature allowed to solidify. 

4. These function has set by default the number of liquid phases; however, multiple solid phases can be computed. Make sure the maximum number of phases is lower or equal to the number of components in the mixture (Gibbs phase rule)

Firstly, the required functions are imported. 

In [1]:
import numpy as np
from phasepy import component, mixture, virialgamma
from phasepy.equilibrium import sle, slle

### Mixture definition 

In this notebook, a ternary mixture of ibuprofen (1) + ethanol (2) + water (3) will be considered.

Ibuprofen will be allowed to solidify. Therefore, its fusion enthalpy `[J/mol]` and temperature `[K]` are supplied.

The mixture will be modelled using the original UNIFAC activity coefficient model.

In [2]:
hf_ibuprofen = 25500 # J/mol
tf_ibuprofen = 347.15 # K
ibuprofen = component(name='ibuprofen', Tc=765., Pc=29.8, Zc=0.313, Vc=668.,
                      GC={'CH3':3, 'CH':1, 'ACH':4, 'ACCH2':1, 'ACCH':1, 'COOH':1},
                      dHf=hf_ibuprofen, Tf=tf_ibuprofen)

water = component(name='water', Tc=647.096, Pc=220.64, Zc=0.229,
                  Vc=55.9472,GC={'H2O':1})

ethanol = component(name='ethanol', Tc=514, Pc=61.37, Zc=0.241, Vc=168.,
                    GC={'CH3':1, 'CH2':1, 'OH':1})

mix = ibuprofen + ethanol + water
eos = virialgamma(mix, virialmodel='ideal_gas', actmodel='original_unifac')

### Solid-Liquid-Equilibria

This phase equilibrium is computed using the `sle` function. The liquid fugacity coefficient is obtained as usual from the activity coefficient model or a cubic equation of state. The solid phase is considered pure, and its fugacity coefficient is computed as follows:

$$ \ln \phi^s = \ln \phi^l - \frac{\Delta H_f}{R} \left(\frac{1}{T} - \frac{1}{T_f}\right)$$

The phase equilibrium is computed with a flash that verifies equilibria and stability between the liquid and solid phases. **Note:** The stability of the liquid phase itself is not verified. The liquid phase can still be unstable. In that case, using the `slle` function is recommended. 

The SLE is computed as follows:
- `beta`: [liquid phase, solid phases] -> phase fractions.
- `tetha`: [solid phases] -> the liquid phase is assumed to be stable. If tetha > 0, it means that the phase is unstable.

In [3]:
P = 1.01325 # bar
T = 318.15 # K
Z = np.array([0.6, 0.3, 0.1])
sle(Z, T, P, eos, solid_phases_index=[0], full_output=True)

            T: 318.15
            P: 1.01325
  error_outer: 4.9841582070805744e-11
  error_inner: 1.0168101651486629e-10
         iter: 11
         beta: array([0.64400275, 0.35599725])
        tetha: array([0.])
      X_fluid: array([[0.37888464, 0.46583652, 0.15527884]])
      v_fluid: [None]
 states_fluid: ['L']
      X_solid: array([[1., 0., 0.]])

### Solid-Liquid-Liquid-Equilibria

Similarly, the `slle` is available if the liquid mixture is unstable. If no initial guesses for the liquid phase are provided, the function uses `lle_init` to generate them.

For the same previous data point, an SLLE can be attempted. In this case, the phase fraction of the second liquid phase is zero, and hence at these conditions, this mixture only exhibits SLE.

In [4]:
P = 1.01325 # bar
T = 318.15 # K
Z = np.array([0.6, 0.3, 0.1])
slle(Z, T, P, eos, solid_phases_index=[0], full_output=True)

            T: 318.15
            P: 1.01325
  error_outer: 4.984148542203448e-11
  error_inner: 1.8121569149448063e-10
         iter: 11
         beta: array([0.64400275, 0.        , 0.35599725])
        tetha: array([0., 0.])
      X_fluid: array([[0.37888464, 0.46583652, 0.15527884],
       [0.37888464, 0.46583652, 0.15527884]])
      v_fluid: [None, None]
 states_fluid: ['L', 'L']
      X_solid: array([[1., 0., 0.]])

The solver can also handle cases where the stable solution includes two liquids and a solid, as shown below:

In [5]:
P = 1.01325 # bar
T = 318.15 # K
Z = np.array([0.40, 0.35, 0.35])
slle(Z, T, P, eos, solid_phases_index=[0], full_output=True)

            T: 318.15
            P: 1.01325
  error_outer: 3.198117594922145e-08
  error_inner: 2.0015495563078728e-10
         iter: 55
         beta: array([0.81623743, 0.16467607, 0.11908649])
        tetha: array([0., 0.])
      X_fluid: array([[3.44030753e-01, 3.92795847e-01, 2.63173400e-01],
       [6.23824634e-04, 1.78443203e-01, 8.20932973e-01]])
      v_fluid: [None, None]
 states_fluid: ['L', 'L']
      X_solid: array([[1., 0., 0.]])

And, if the two liquids split is more stable than the three-phase split, the solver handles that automatically. As can be seen, the phase fraction of the solid phase is zero, and its stability variable is positive.

**Note**: `error_outer` corresponds to the error in the successive substitution or norm of Jacobian obtained from minimizing the Gibbs free energy. If one of the phases has been eliminated during the Gibbs free energy minimization, then the norm of this Jacobian can be far from zero. (As is the case below).

In [6]:
P = 1.01325 # bar
T = 318.15 # K
Z = np.array([0.2, 0.4, 0.4])
slle(Z, T, P, eos, solid_phases_index=[0], full_output=True)

            T: 318.15
            P: 1.01325
  error_outer: 0.08833939563934295
  error_inner: 3.5659831981451483e-10
         iter: 26
         beta: array([0.73696836, 0.26303164, 0.        ])
        tetha: array([0.        , 0.10771003])
      X_fluid: array([[0.27058093, 0.45127219, 0.27814687],
       [0.00224465, 0.25634434, 0.74141102]])
      v_fluid: [None, None]
 states_fluid: ['L', 'L']
      X_solid: array([[1., 0., 0.]])

For further information please also check [official documentation](https://phasepy.readthedocs.io/), or just try:

```function?```